#Paquetes necesarios

# TRAIN YOLO

In [15]:
from abc import abstractmethod, ABC
from numpy import array

class PlateDetector(ABC):
    @abstractmethod
    def detect(self, img: array) -> list[array]:
        pass

class PlateMatcher(ABC):
    @abstractmethod
    def match(self, text : str) -> bool:
        pass

class TextProcessor(ABC):
    @abstractmethod
    def process(self, text : str) -> str:
        pass

class TextExtractor(ABC):
    def __init__(self, matcher : PlateMatcher, processor : TextProcessor):
        self.matcher = matcher
        self.processor = processor

    @abstractmethod
    def extract(self, img: array) -> str:
        pass

In [16]:
class ImageProcessor(ABC):
    @abstractmethod
    def process(self, img: array) -> array:
        pass

class CannyImageProcessor(ImageProcessor):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def process(self, img: array) -> array:
        #apply Canny
        import cv2 as cv
        canny = cv.Canny(img, self.x, self.y)
        return canny

class SobelImageProcessor(ImageProcessor):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def process(self, img: array) -> array:
        #apply Sobel
        import cv2 as cv
        sobel = cv.Sobel(img, cv.CV_8U, self.x, self.y, ksize=5)
        return sobel

In [17]:
class YOLOPlateDetector(PlateDetector):
    def __init__(self, model, category = 0, image_processors = list()):
        self.model = model
        self.category = category
        self.image_processors = image_processors

    def detect(self, img):
        from ultralytics import YOLO
        model = YOLO(self.model)
        results = model(img)
        cars = []
        for x in results:
            boxes = x.boxes
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                category = int(box.cls[0])
                if category == 0:
                    img = img[y1:y2, x1:x2]
                    for x in self.image_processors:
                        img = x.process(img)
                    cars.append(img)
        return cars

In [18]:
class LengthPlateMatcher(PlateMatcher):
    def match(self, text : str) -> bool:
        if len(text) == 7: return True
        return False

class RegexPlateMatcher(PlateMatcher):
    def match(self, text : str) -> bool:
        import re
        pattern = re.compile("^[0-9]{4}([B-D]|[F-H]|[J-N]|[P-T]|[V-Z]){3}$")
        if pattern.match(text):
            return True
        else:
            return False

In [19]:
class TesseractTextExtractor(TextExtractor):
    def extract(self, img):
        import pytesseract
        pytesseract.pytesseract.tesseract_cmd = r'/usr/local/bin/tesseract'
        text = pytesseract.image_to_string(img)
        matcher = RegexPlateMatcher()
        if matcher.match(text):
            text = self.processor.process(text)
            return text
        
class EasyOCRTextExtractor(TextExtractor):
    def extract(self, img):
        import easyocr
        reader = easyocr.Reader(['es']) 
        result = reader.readtext(img)
        if(len(result) == 0): return ""
        for x in result:
            text = x[1]
            text = self.processor.process(text)
            if(self.matcher.match(text)):
                return text

class KerasOCRTextExtractor(TextExtractor):
    def extract(self, img):
        from keras_ocr.detection import Detector
        from keras_ocr.recognition import Recognizer
        from keras_ocr import pipeline

        detector = Detector()
        recognizer = Recognizer()
        pipeline = pipeline.Pipeline(detector=detector, recognizer=recognizer)
        prediction_groups = pipeline.recognize([img])
        for group in prediction_groups:
            for word in group:
                text = word[0]
                text = self.processor.process(text)
                if(self.matcher.match(text)):
                    return text

In [20]:
class BasicTextProcessor(TextProcessor):
    def process(self, text : str) -> str:
        text = text.replace(" ", "")
        text = text.replace("-", "")
        return text

In [25]:
class CarPlateDetector:
    def __init__(self,  text_extractor : TextExtractor, plate_detector : PlateDetector):
        self.plate_detector = plate_detector
        self.text_extractor = text_extractor

    def detect(self, image : str) -> list[str]:
        import cv2 as cv
        image = cv.imread(image)
        cars = self.plate_detector.detect(image)
        return [self.text_extractor.extract(x) for x in cars]

In [22]:

class ColorThresholdImageProcessor(ImageProcessor):
    def __init__(self, lower, upper):
        self.lower = lower
        self.upper = upper
    
    def process(self, img: array) -> array:
        import cv2 as cv
        img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        mask = cv.inRange(img, self.lower, self.upper)
        img = cv.bitwise_and(img, img, mask=mask)
        return img

class ContoursImageProcessor(ImageProcessor):
    def process(self, image: array) -> array:
        import cv2
        import numpy as np

        mask = np.ones(image.shape, dtype=np.uint8) * 255
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
        dilate = thresh

        cnts = cv2.findContours(dilate, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        for c in cnts:
            area = cv2.contourArea(c)
            if area < 300:
                x,y,w,h = cv2.boundingRect(c)
                mask[y:y+h, x:x+w] = image[y:y+h, x:x+w]
        return mask

In [28]:
text_extractor=EasyOCRTextExtractor(
    processor=BasicTextProcessor(), 
    matcher=LengthPlateMatcher()
)


plate_detector = YOLOPlateDetector(model='plate_recognizer.pt', image_processors=[
    ]
)

detector = CarPlateDetector(
    text_extractor,
    plate_detector
)


def  detectImage():
    result = detector.detect('./plates/coches2.jpg')
    print(result)

for x in range(0, 10):
    detectImage()


0: 384x640 1 License_Plate, 142.2ms
Speed: 9.8ms preprocess, 142.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)


['0478LFR']



0: 384x640 1 License_Plate, 109.2ms
Speed: 2.9ms preprocess, 109.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)


['0478LFR']



0: 384x640 1 License_Plate, 126.2ms
Speed: 3.5ms preprocess, 126.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)


...

['0478LFR']



0: 384x640 1 License_Plate, 107.8ms
Speed: 3.1ms preprocess, 107.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)


['0478LFR']



0: 384x640 1 License_Plate, 107.8ms
Speed: 2.6ms preprocess, 107.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)


['0478LFR']


In [29]:
import threading 

text_extractor=EasyOCRTextExtractor(
    processor=BasicTextProcessor(), 
    matcher=LengthPlateMatcher()
)

plate_detector = YOLOPlateDetector(model='plate_recognizer.pt')

detector = CarPlateDetector(
    text_extractor,
    plate_detector
)


def  detectImage():
    result = detector.detect('./plates/coches2.jpg')


threads = [
]

def add_threads(num_threads=10):
    for x in range(0, num_threads):
        threads.append(threading.Thread(target=detectImage))
    
def join_threads():
    for x in threads:
        x.start()
    for x in threads:
        x.join()

add_threads(10)
join_threads()




0: 384x640 1 License_Plate, 242.1ms
Speed: 4.4ms preprocess, 242.1ms inference, 14.9ms postprocess per image at shape (1, 3, 384, 640)





0: 384x640 1 License_Plate, 195.7ms
Speed: 4.3ms preprocess, 195.7ms inference, 10.0ms postprocess per image at shape (1, 3, 384, 640)




0: 384x640 1 License_Plate, 250.6ms
Speed: 3.4ms preprocess, 250.6ms inference, 5.9ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 1 License_Plate, 260.2ms
Speed: 3.0ms preprocess, 260.2ms inference, 9.6ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 1 License_Plate, 268.1ms
Speed: 8.9ms preprocess, 268.1ms inference, 9.7ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 1 License_Plate, 282.4ms
Speed: 5.6ms preprocess, 282.4ms inference, 18.8ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 1 License_Plate, 265.1ms
Speed: 4.5ms preprocess, 265.1ms inference, 8.5ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 1 License_Plate, 250.2ms
Speed: 8.7ms 

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2'

In [37]:
import cv2
import time

image = cv2.imread('./plates/coches2.jpg')
processor = BasicTextProcessor()
# matcher = LengthPlateMatcher()
matcher = RegexPlateMatcher()
detector = YOLOPlateDetector(model='plate_recognizer.pt')
extractor = EasyOCRTextExtractor(matcher, processor)
start = time.time()
plates = detector.detect(image)
print("plate detection time:", time.time() - start, "seconds")

# Dibujar rectángulos alrededor de las placas detectadas
for plate in plates:
    if len(plate) == 5:  # Asegurarse de que haya cinco valores en la tupla
        print(plate)
        x1, y1, x2, y2, img = plate
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        start = time.time()
        detection = extractor.extract(img)
        print("easyocr reading time:", time.time() - start, "seconds")
        if detection == None or detection == '': detection = "unknown plate"
        cv2.putText(image, str(detection), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Mostrar la imagen con los rectángulos dibujados
cv2.imshow('Plates Detection', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


0: 384x640 1 License_Plate, 105.6ms
Speed: 2.5ms preprocess, 105.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)


plate detection time: 1.5460870265960693 seconds


In [30]:
import cv2

cap = cv2.VideoCapture('videos/license_plates_fps.mp4')
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) / 2)
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) / 2)

processor = BasicTextProcessor()
# matcher = LengthPlateMatcher()
matcher = RegexPlateMatcher()
detector = YOLOPlateDetector(model='plate_recognizer.pt')
extractor = EasyOCRTextExtractor(matcher, processor)

while True:
    ret, frame = cap.read()
    if not ret: break

    frame = cv2.resize(frame, (width, height))
    plates = detector.detect(frame)

    # Dibujar rectángulos alrededor de las placas detectadas
    for plate in plates:
        if len(plate) == 5:  # Asegurarse de que haya cuatro valores en la tupla
            x1, y1, x2, y2, img = plate
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            detection = extractor.extract(img)
            if detection == None or detection == '': detection = "unknown plate"
            cv2.putText(frame, detection, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow('Plates', frame)
    if cv2.waitKey(20) == 27: 
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 2 License_Plates, 109.6ms
Speed: 3.4ms preprocess, 109.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 License_Plate, 101.7ms
Speed: 3.2ms preprocess, 101.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 License_Plate, 97.6ms
Speed: 2.4ms preprocess, 97.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 License_Plate, 96.4ms
Speed: 2.4ms preprocess, 96.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 License_Plate, 96.2ms
Speed: 2.7ms preprocess, 96.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 License_Plate, 94.9ms
Speed: 2.7ms preprocess, 94.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 License_Plate, 105.5ms
Speed: 2.8ms preprocess, 105.5ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 License_Plate, 103.3ms
Speed: 2.3ms preprocess, 

KeyboardInterrupt: 